# Creating a convention using [metadata4ing](https://nfdi4ing.pages.rwth-aachen.de/metadata4ing/metadata4ing/#http://w3id.org/nfdi4ing/metadata4ing#ContactPerson)

In [1]:
import h5rdmtoolbox as h5tbx
from h5rdmtoolbox import convention

Failed to import module h5tbx


In [2]:
cv = convention.Convention(name='PersonConvention', contact='John Doe')
cv.register()

In [3]:
from pydantic import BaseModel
from typing_extensions import Literal

Role = Literal[
    "Creator",
    "ContactPerson",
]

class Person(BaseModel):
    first_name: str
    last_name: str
    role: Role

class SpecialStandardAttribute(h5tbx.convention.standard_attributes.StandardAttribute):

    def set(self, parent, value, attrs=None):
        """Is called when attribute of a standard attribute is set"""
        print(value)
        grp_name = f'{value["role"]}Person'
        i = 0
        while grp_name in parent:
            i += 1
            grp_name = f'{value["role"]}Person{i}'
        person_grp = parent.create_group(grp_name)
        
        person_grp.create_string_dataset('first_name',
                                         data=value['first_name'],
                                        attrs=dict(iri='http://xmlns.com/foaf/0.1/firstName',
                                                  description='The first name of a person'))
        person_grp.create_string_dataset('last_name',
                                         data=value['last_name'],
                                        attrs=dict(iri='http://xmlns.com/foaf/0.1/lastName',
                                                  description='The last name of a person'))

personStdAttr = SpecialStandardAttribute(
    name='person',
    validator=Person,
    target_method='__init__',
    description='Person',
    default_value='$empty'
)

In [4]:
cv.add_standard_attribute(personStdAttr)
cv.registered_standard_attributes

{'person': <SpecialStandardAttribute@__init__[positional/obligatory]("person"): "Person.">}

In [5]:
h5tbx.use(None)
h5tbx.use(cv.name)

using("PersonConvention")

In [6]:
Person(first_name='Matthias', last_name='Probst', role='Creator')

Person(first_name='Matthias', last_name='Probst', role='Creator')

In [7]:
with h5tbx.File(person=dict(first_name='Matthias', last_name='Probst', role='Creator')) as h5:
    h5.dump()

{'first_name': 'Matthias', 'last_name': 'Probst', 'role': 'Creator'}
